<a href="https://colab.research.google.com/github/Thelmaandherls/AI-ML/blob/Udemy/Deep_Q_Learning_for_Lunar_Landing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


### Importing the libraries

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [ ]:
class Brain(nn.Module):

  def __init__(self, state_size, action_size, seed = 42): #state_size is 8 but will be speciiefd later in trianining, action size is 4 and seed is 42
      super(Brain, self).__init__() #aim of this row is to activate the inheritance
      self.seed = torch.manual_seed(seed) #geenrate vectors
      self.fcl1 = nn.Linear(state_size, 64) #first full connetion between input layer ad the first fully conencted later- state_size = #neurons in the input layer and 64= optimum number of neurons in the first fully connected layer to build a fully fucntional AI via actual calcs
      self.fcl2 = nn.Linear(64, 64) #if i was to have just 1 fcl then the #would be 4 as that si the numebr of the output, but to have another fcl, the best #neruons si 64
      self.fcl3 = nn.Linear(64, action_size) #4 = action size

  #building the forward method inside the Brain class which will forward-propaagte the signal from input to output layers through both fcl
  def forward(self, state): #called it state as it's going to forward-prop a sig from the state to the output layer to playy a specific aciton at a specific time
      x = self.fcl1(state) #propagate sig from input to fcl1 - use fcl1 obj as it's an insatcne fo the input class and retirn the first full conenction - it takes the state as a current time step in the env - as fc1 state returns the first ully conencted layer, create a new var x to contain what is returned by self.fc1(state)
      x = F.relu(x) #activate the sig thanks to a rectifier activation function by updating the value of x but calling first the F short from the improted functional pytorch module and call the reluc func which is the rectifier activation func
      #these 2 lines of progapte the sig from the input to the fc1 with a rectifier activation func and then do the same from the fcl2
      x = self.fcl2(x) #fc2 takes as input the sig from the fc1 so use x and not state
      x = F.relu(x)
      return self.fcl3(x)
      #final sig in the output layer containing actions
      #this ofrward props the states form input to output
      #pytorch can create the architcture efficiently

## Part 2 - Training the AI

### Setting up the environment

initialising vars

In [ ]:
import gymnasium as gym
env = gym.make('LunarLander-v2') #import env ai will be trained
#paramteres I will req later for training
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0] #number of elements at easch inputs - the cordinates describing the states
num_actions = env.action_space.n
print("State shape:", state_shape) #vector of 8 elements
print("State size:", state_size) # 8 input values in each state
print("Number of actions:", num_actions) # 4 actiosn that can be executed

State shape: (8,)
State size: 8
Number of actions: 4


### Initializing the hyperparameters

In [ ]:
learning_rate = 5e-4 #best value after exps
minibatch_size = 100 #num of osebcs used in 1 step of the trianing to update mdoel parameters - use this as the usual value for DQL
discount_factor = 0.99 #represents values of rewards - close to 1 = better cnsideration of future rewards
replay_buffer_size = int(1e5) #size of the memroy of the ai- the number of exepriences, rewafds, mixed states and action performed - toimprve trianing process
interpolation_parameter = 1e-3 #interpolation prameter used for training - used for sub update - best value after exp

### Implementing Experience Replay

In [ ]:
class ReplayMemory(object):
  #use this class name typically for class that inplements expeienrce replay and use noinheritance as use bject
  def __init__(self, capacity):
      self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #usful if want to make faster and moreperformant usogn a gpu
      self.capacity = capacity #max size of memory buffer
      self.memory = [] #list that will store expeience, each one including state, reward, next state done ornot - initlaise this to an exmpty list
      #this si the ocntrutor method that initalises the replay memory obj

#adding a new method into the replay memery class which will add new expeirnce into the replay memeory class without overlaoding the capcity
#experience = current state, fuure state, reward, action, boolean saying if action is done or not
  def push(self, experience):
    self.memory.append(experience) #append an event to this memory
    #ensure ememry buffer does not exceed capcity
    if len(self.memory) > self.capacity:
      del self.memory[0] #removing the oldest memory

  #sample method that will randomly selct a batch of experiences from the memory buffer

  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    #going to extract the diff elemts of eahc smaple fo expeience indiviually
    #1 row for loop = for loop within a list ([]) - then stack the first elemtns within those experiences as long as they exist
    #convert these stack of states into pytorch tensors via torch library = torch.from_numpy()
    #ensure that the datate type is float to conserve the format expected by pytorch
    #.to(Self.device) = to send back to the designated computign device
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device) #.long = long int as action can be 0,1,2,3
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device) #uint8 converts to bool which si then converted to float
    return states, next_states, actions, rewards, dones




### Implementing the DQN class

creating an agent class that defs behvaiour of agent that inetrcat with space env via dqn - whislt interacting w env, agent maintian 2 nws: local and target
locla = selects action
targte = calcs taregt q values that will be used to trian local qnw
this double 1nw setup will staebiise learning process in learn method
softupdate method will update target qnw params by blending them with those of the locla q nw using the softupdate formuale - this prevents abrupt changes that could destabilsie the trianing
implemented act agent that wil help agent choose actions based on current understanfinf of optimal policty -will be returned from local qnw that will fwd prop the state to obtain action values and then use the epsiolon greedy policty to resturn final action
eg strategy is used as expiration mech - fact tahts oemtimes can selct random actions enables agent to explore more actions that could possibly eb more beneficial
learn method uses experiences samples from replay ememory to update local qnw q values towards the target qnw

In [ ]:
class Brainiac():

  def __init__(self, state_size, action_size):
    #choose if want to use the cpu or gpu
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #defining these vars according ot the obj var and initlaising them to the argument that wiull be entered later on when creaitng an obj in the instnce of this brainian class
    self.state_size = state_size
    self.action_size = action_size #size of action space
    #doing Q-learning - creating local and target networks
    #create 2 instance of previously madenetwarok - 1 that will be local and the other will eb target
    self.local_qnetwork = Brain(state_size, action_size).to(self.device)
    self.target_qnetwork = Brain(state_size, action_size).to(self.device)
    self.optimiser = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate) #will update step by step to optimsie actions - create as nstacne of adam clas that will enbale us to use the  adam optimiser and call an attribute of the local qnetwork, parmaters
    #to implement the dql class
    self.memory = ReplayMemory(replay_buffer_size) #replay_buffer_size = capcity
    self.t_step = 0 #counter

#method to store expeirneces and decdie when to learn from them
  def step(self, state, action, reward, next_state, done):
    #store experience in replay memeory
    self.memory.push((state, action, reward, next_state, done))
    #increment the time stem counter sekf.t_step and reset every 4 steps so we can leanr every 4 step s
    self.t_step = (self.t_step + 1) % 4
    #learning which will happen every 4 steps - learn base don mini batches
    if self.t_step == 0:
      #check if #of expeirences in memory is already larger than mini batch size fof 100
      #using self.memory instance of the replay memory class and calling the atrribute self.memory from the replaymemory class
      if len(self.memory.memory) > minibatch_size:
        #using smaple method from replay memory class which will sample 100 exps from the memory
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

#method to select an action based on a given state and a certain epislon value for an epislon greedy action selction policy
  def act(self, state, epsilon = 0.):
    #state os a numpy array so want it to be a torch tesnor to work with nn
    #use .unsqueeze - add an extra dimension to state vector - currently have 8 dimensions coresponding to coords os spaceship
    #this dimension says which batch the state belongs to for each state
    #we're using the index of 0 so the first dimension of new state vector / now torch tensor will be at beginning
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    #forward pass state through local  network to get action values, goign to set locla q network to evalaution mode
    self.local_qnetwork.eval()
    #ensure in inference mode (make predictions) - use no_grad to make sure any gradient computation is disabled
    with torch.no_grad():
      #make the predictions from the lcoal q network to get the action values/q values corresponding to actions
      action_values = self.local_qnetwork(state)
    #going back into training mode via .train()
    self.local_qnetwork.train()
    #epsilon greedy action selection
    if random.random() > epsilon:
      #select action with highest q value and in numpy array
      return np.argmax(action_values.cpu().data.numpy())
    else:
      #will select a random number between index 0-3
      return random.choice(np.arange(self.action_size))

#learn memthod to update agents q values based on sample expereinces
  def learn(self, experiences, discount_factor):
    #unpack smaple experiences into theit repective vars - upacking the tuple that makes up the experiences
    states, next_states, actions, rewards, dones = experiences
    #get max predicted q value fron next_state from target nn/propaagting the next state from target qnw
    #because we want to compute q targets for the current state which req maxnq values from next state in formula
    #use max of action vaues propapagted through next states - need max value along dimension 1 which corrsponds to action value along the tesor
    #after getting max 1 we get 2 tensors: tesnor of max values and tensor of indivies corrsponding to actiosn of these max values
    #.unqueeze - need to add imenrion of the batch at position 1
    #detach- detaches resulting resnor from computation graph so won't be tracking gradient from backpropagatoon
    #this iverall gives the max currated q values from the next states from the target network
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    #computing q taregts for the current states - formual that is used often
    q_targets = rewards + (discount_factor * next_q_targets * (1 - dones))
    #forward prop the states from the local q network/expected q values from local qnetwork
    q_expected = self.local_qnetwork(states).gather(1, actions)
    #compute loss between expected and target q values - mean swqusred error losss
    loss = F.mse_loss(q_expected, q_targets)
    #initalise the optimiser by restting it / making it 0
    self.optimiser.zero_grad()
    #back prop the loss to compute the gradient of the closs with respoect to the model params
    #this update model params, to update q values -> better action slection poliity
    loss.backward()
    #singlpe optimisation step to update model params
    self.optimiser.step()
    #update target nw params with thoseof the local nn
    #always use interpolation para, fro soft update
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

#implement soft_updatemethod- method that ill softly update the target network's aparm
#soft update: softly updating the target model params using weight avg of local and target params
  def soft_update(self, local_model, target_model, interpolation_parameter):
    #loop through oarams of local and target
    #zip = get params of local and target at the same time
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      #use .copy func to update pparams of target qnw - in brackets = formula of softupdate fo target param
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)


### Initializing the DQN agent

creatign an instance of the branianc class
not inputting action_size as when intialled the vars in the setting up env, it was aclled number of actions

In [ ]:
brainiac = Brainiac(state_size, num_actions)

### Training the DQN agent

In [ ]:
#initlais etrianing params
#max num of epsidoes we want to train agent but tend to get good reward from abotu a couple 100s
num_epsiodes = 2000
#max num of time steps per episode - if ai is staying stuck then it can restart
maximum_num_timesteps_per_episode = 1000
#series of 4 corrsponding to hyepr params correlating to epsiolon greddy action selction policity
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
#decrement - multplyinh it by very close to 1 flaot soit decays by very little  until reach 0.1
epsilon_decay_rate = 0.995
epsilon = epsilon_starting_value
#window of scores of 100 epsidoe - initlaise with a double ended q
scores_on_100_eps = deque(maxlen = 100)

#step where ai has specific intelligence to land on the moon
#upper boudn fo range is excluded in py so add 1 to include it
for episode in range(1, num_epsiodes + 1):
  #always inatrinaing loop, first step is to reset env to intial state at every ep
  #.rest returns intial state and other info e.g. intial observation which is discarded by the undeerscore
  state, _ = env.reset()
  #initlaise the score/ accumualted reward
  score = 0
  #looping time steps over the eps
  for t in range(maximum_num_timesteps_per_episode):
    #slecting an action - use .act to slect action in agiven state of env follwing eg policy
    action = brainiac.act(state, epsilon)
    #making a new move -> ai ends in new state and therefore needs a reward
    next_state, reward, done, _, _ = env.step(action)
    #one of th emost important steps
    #tool that will eprform the training
    #step method was implemented in agent claass which includes the leanr memehtod
    #learn method performs trianing by back prop loss b/w expcted q and target q values with optimiser and on smaples experiences
    #this line of code trians agent to land better on the moon
    brainiac.step(state, action, reward, next_state, done)
    #update state var so it becomes next stae
    state = next_state
    #score isaccumualted score
    score += reward
    #check if ep is done
    if done:
      break
  #score of last ep just finished
  scores_on_100_eps.append(score)
  #decay eg value incrementally whilst ensuring ti doesnt' go beyond ending eg
  epsilon = max(epsilon_ending_value, epsilon_decay_rate * epsilon)

#getting the avg score for each ep in dynamic way
#ep by ep will see avg of accumualted reward evolving over the eps
#but make over wrting effect so each line printed will be removed to give place to the enxt one
#retain avg reward over 100 eps after every 100 eps
#\ - adds space
#tAverage score = accumualtive reward over last 100 eps unless havent reach 100s eps thne from 0 to this one
#:.2f - want avg to be rounded to 2 decimal places
#episode = #eps
#avg score of var = np.mean(scores_on_100_eps) - as long as haven't rached 100 eps, the scores_on_100_epswill contain less than 100 eps and thf get mean of <100 eps
#\r = carriage returing programme- cursor willr etunr to start of the line when prinitng so in a loop, it will allow newly printed line to override previous -> dynamic effect in console
#end = after printing this lien, won't automaticlly create new line which is vital to retain the dyamic programming of \r
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_eps)), end = "")
  #check if we are every 100 epsidode using modulo operator to check the rest of the euclean division is 0
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_eps)))
  #ep = solution if scores at least 200 points = winning morale - form lunar docu
  if np.mean(scores_on_100_eps) >= 200.0:
    #:d = will be double inter
    #espisode - 100 - started winiing from this number - 100 as this is the score over 100 eps - could keep as just episodes
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_eps)))
    #save it's params
    #.state_dict = method that will retunr us a dictionary of the model params
    torch.save(brainiac.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -159.36
Episode 200	Average Score: -117.98
Episode 300	Average Score: -26.86
Episode 400	Average Score: 104.15
Episode 500	Average Score: 174.21
Episode 600	Average Score: 95.62
Episode 700	Average Score: 150.17
Episode 800	Average Score: 182.98
Episode 814	Average Score: 200.53
Environment solved in 714 episodes!	Average Score: 200.53


## Part 3 - Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(brainiac, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = brainiac.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(brainiac, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()